# Example of Ask/Tell interface with DEAP 

In [1]:
import random
import array
import numpy

from deap import base, benchmarks, creator, tools, algorithms
from deap import cma

In [2]:
import numpy as np
from brian2 import *
from brian2.equations.equations import (DIFFERENTIAL_EQUATION, Equations,
                                        SingleEquation, PARAMETER)
from brian2.input import TimedArray
from brian2 import NeuronGroup, StateMonitor, store, restore, run, defaultclock, second, Quantity
from brian2.stateupdaters.base import StateUpdateMethod

### Input

In [3]:
input_traces = zeros((10,1))*volt
for i in range(1):
    input_traces[1:,i]=i*10*mV

In [4]:
# Create target current traces
output_traces = 10*nS*input_traces

In [5]:
input = input_traces
output = output_traces

In [6]:
params = np.array([
 [ 1.80869973e-08,  2.50218013e-02],
 [ 1.88373085e-08,  9.89559934e-02], 
 [ 1.88373085e-08,  9.89559934e-02], 
])

### Setup The Model for Optimization

In [7]:
input_var = 'v'
output_var = 'I'

parameter_names = {'g', 'E'}
method = ('linear', 'exponential_euler', 'euler')
t_start = 0*second
popsize, _ = np.shape(params)
dt = 0.1 *ms
defaultclock.dt = dt

In [8]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [9]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

INFO       No numerical integration method specified, using method 'linear' (took 0.00s). [brian2.stateupdaters.base.method_choice]


### Required Model Operations

In [10]:
Nsteps, Ntraces = input_traces.shape
# N = popsize * len(parameter_names)
N = popsize
duration = Nsteps*dt

In [11]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [12]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

In [13]:
neurons = NeuronGroup(Ntraces*N, model, method = method)
neurons.namespace['input_var'] = input_traces
neurons.namespace['output_var'] = output_traces
neurons.namespace['t_start'] = t_start
neurons.namespace['Ntraces'] = Ntraces

#### Record error  
additional differential equation calculating the error

In [14]:
neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                      when='end')

# Add the code doing the numerical integration
neurons.run_regularly(state_update_code, when='groups')

# store the state of the network
store()

In [15]:
def parameters_dict(params):
    d = dict()
    for name, value in zip(parameter_names, params.T):
        d[name] = value
            
    return d

In [16]:
def calc_error(params):
    print(params)
    popsize, _ = np.shape(params)
    N = popsize

#     neurons = NeuronGroup(Ntraces*N, model, method = method)
    neurons = NeuronGroup(N, model, method = method)
    neurons.namespace['input_var'] = input_traces
    neurons.namespace['output_var'] = output_traces
    neurons.namespace['t_start'] = t_start
    neurons.namespace['Ntraces'] = Ntraces

    # Record error
    neurons.run_regularly('total_error +=  (' + output_var + '-output_var(t,i % Ntraces))**2 * int(t>=t_start)',
                          when='end')

    # Add the code doing the numerical integration
    neurons.run_regularly(state_update_code, when='groups')

    d = parameters_dict(params)
    neurons.set_states(d, units=False)
    run(duration, namespace = {})

    e = neurons.total_error/int((duration-t_start)/defaultclock.dt)
    e = mean(e.reshape((N,Ntraces)),axis=1)
    
    return array(e)

In [17]:
neurons

NeuronGroup(clock=Clock(dt=100. * usecond, name='defaultclock'), when=start, order=0, name='neurongroup')

### Ask and Tel with calc_error Function

In [18]:
start_scope()

In [19]:
model = Equations('''
I = g*(v-E) : amp
g : siemens (constant)
E : volt (constant)
''')

In [20]:
state_update_code = StateUpdateMethod.apply_stateupdater(model, {}, method=method)

In [21]:
model_without_diffeq = Equations([eq for eq in model.ordered
                                      if eq.type != DIFFERENTIAL_EQUATION])
    
# Add a parameter for each differential equation
diffeq_params = Equations([SingleEquation(PARAMETER, varname, model.dimensions[varname])
                           for varname in model.diff_eq_names])

# Our new model:
model = model_without_diffeq + diffeq_params

# Replace input variable by TimedArray
input_traces = TimedArray(input, dt = dt)

In [22]:
input_unit = input.dim
model = model + Equations(input_var + '= input_var(t,i % Ntraces) : '+ "% s" % repr(input_unit))

# Add criterion with TimedArray
output_traces = TimedArray(output, dt = dt)
error_unit = output.dim**2
model = model + Equations('total_error : %s' % repr(error_unit))

### set up DEAP

In [23]:
NDIM = 2
IND_SIZE = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

strategy = cma.Strategy(centroid=[2.0]*NDIM, sigma=1.0)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, -2, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)

toolbox.register("generate", strategy.generate, creator.__dict__["Individual"])
toolbox.register("update", strategy.update)

In [24]:
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)

In [25]:
stats = None
ngen = 1

logbook = tools.Logbook()
logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

for gen in range(ngen):
    # Generate a new population
    population = toolbox.generate()
    print(population, '\n')

[[2.485379132140227, 1.9751958155600446], [3.822314163940711, 1.3783809467340231], [0.9261902988489363, 2.638165974093604], [1.4170675739090917, 2.6304884975086313], [1.1320978206327952, 0.14415473882025798], [1.3520826403293011, 2.4327696133505143]] 



In [26]:
population

[[2.485379132140227, 1.9751958155600446],
 [3.822314163940711, 1.3783809467340231],
 [0.9261902988489363, 2.638165974093604],
 [1.4170675739090917, 2.6304884975086313],
 [1.1320978206327952, 0.14415473882025798],
 [1.3520826403293011, 2.4327696133505143]]

In [27]:
fitnesses = calc_error(np.array(population))

for ind, fit in zip(population, fitnesses):
    print(ind, fit)
    ind.fitness.values = (fit,)

[[2.48537913 1.97519582]
 [3.82231416 1.37838095]
 [0.9261903  2.63816597]
 [1.41706757 2.6304885 ]
 [1.13209782 0.14415474]
 [1.35208264 2.43276961]]
[2.485379132140227, 1.9751958155600446] 24.09936552697533
[3.822314163940711, 1.3783809467340231] 27.758198814699007
[0.9261902988489363, 2.638165974093604] 5.9704172712489925
[1.4170675739090917, 2.6304884975086313] 13.894852308693785
[1.1320978206327952, 0.14415473882025798] 0.026633347516388635
[1.3520826403293011, 2.4327696133505143] 10.819531081071569


In [28]:
hof.update(population)

In [29]:
toolbox.update(population)

In [30]:
record = stats.compile(population) if stats is not None else {}
logbook.record(gen=gen, nevals=len(population), **record)

print(logbook.stream)

gen	nevals
0  	6     


In [31]:
hof[0]

[1.1320978206327952, 0.14415473882025798]

In [32]:
hof[0].fitness.values[0]

0.026633347516388635

In [33]:
print("Best individual is ", hof[0], hof[0].fitness.values[0])

Best individual is  [1.1320978206327952, 0.14415473882025798] 0.026633347516388635


### Different Approach

In [34]:
FITCLSNAME = "FIT_TYPE"
INDCLSNAME = "IND_TYPE"
creator.create(FITCLSNAME, base.Fitness, weights=(-1.0,))
creator.create(INDCLSNAME, list, fitness=creator.__dict__[FITCLSNAME])
# # HV_THRESHOLD = 116.0        # 120.777 is Optimal value

In [35]:
NDIM = 2
BOUND_LOW, BOUND_UP = 0.0, 1.0
MU, LAMBDA = 10, 10
NGEN = 1

In [36]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, -2, 2)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)

WARNING    /home/alteska/anaconda3/envs/brian_opt/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
 [py.warnings]
WARNING    /home/alteska/anaconda3/envs/brian_opt/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
 [py.warnings]


In [37]:
# The MO-CMA-ES algorithm takes a full population as argument
population = [creator.__dict__[INDCLSNAME](x) for x in numpy.random.uniform(BOUND_LOW, BOUND_UP, (MU, NDIM))]


In [38]:
strategy = cma.StrategyMultiObjective(population, sigma=1.0, mu=MU, lambda_=LAMBDA)

toolbox.register("generate", strategy.generate, creator.__dict__[INDCLSNAME])
toolbox.register("update", strategy.update)

In [39]:
for gen in range(NGEN):
    # Generate a new population
    population = toolbox.generate()

In [40]:
# Evaluate the individuals
fitnesses = calc_error(np.array(population))

for ind, fit in zip(population, fitnesses):
    print(ind, fit)
    ind.fitness.values = (fit,)

[[-1.71472196  2.75572225]
 [-0.48956329  1.92070373]
 [ 0.80128125 -0.86417712]
 [ 0.03292285  1.82248562]
 [ 1.56992212 -0.23667107]
 [-1.53884063  2.17151532]
 [ 0.53852833 -0.38167782]
 [ 0.88019078  0.4239663 ]
 [-1.23673975  2.80486175]
 [-1.10710047  2.02427281]]
[-1.7147219579636168, 2.7557222480056254] 22.328435978729523
[-0.4895632938253772, 1.9207037313109836] 0.8841754585706412
[0.8012812450317278, -0.8641771225114863] 0.479485507714806
[0.03292284578102811, 1.8224856159035034] 0.0036001695464601717
[1.569922116025487, -0.23667107263767384] 0.1380532303515676
[-1.5388406280843248, 2.171515318728243] 11.166397471182432
[0.5385283320019196, -0.38167782247122195] 0.04224846793545355
[0.8801907791270561, 0.42396630031872234] 0.139256765557156
[-1.2367397493613113, 2.8048617521423607] 12.03315634655405
[-1.1071004650637448, 2.0242728143620905] 5.022409868530988


In [41]:
# # Update the strategy with the evaluated individuals
# toolbox.update(population)

### Working Bounds?

In [42]:
NDIM = 2
IND_SIZE = 5

creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

strategy = cma.Strategy(centroid=[0.0]*NDIM, sigma=0.0)

toolbox = base.Toolbox()
toolbox.register("attr_int", np.random.uniform, 0, 1)
toolbox.register("attr_flt", np.random.uniform, 0, 2)
toolbox.register("individual", tools.initCycle, creator.Individual,
             (toolbox.attr_int, toolbox.attr_flt),
             n=1)

toolbox.register("generate", strategy.generate, creator.__dict__["Individual"])
toolbox.register("update", strategy.update)

In [43]:
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)

In [44]:
stats = None
ngen = 1

logbook = tools.Logbook()
logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

for gen in range(ngen):
    # Generate a new population
    population = toolbox.generate()
    print(population, '\n')

[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]] 



In [45]:
population

[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0]]

In [46]:
fitnesses = calc_error(np.array(population))

for ind, fit in zip(population, fitnesses):
    print(ind, fit)
    ind.fitness.values = (fit,)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[0.0, 0.0] 0.0
[0.0, 0.0] 0.0
[0.0, 0.0] 0.0
[0.0, 0.0] 0.0
[0.0, 0.0] 0.0
[0.0, 0.0] 0.0


In [47]:
hof.update(population)

In [48]:
toolbox.update(population)

ZeroDivisionError: float division by zero

In [ ]:
record = stats.compile(population) if stats is not None else {}
logbook.record(gen=gen, nevals=len(population), **record)

print(logbook.stream)

In [ ]:
hof[0]

In [ ]:
hof[0].fitness.values[0]

In [ ]:
print("Best individual is ", hof[0], hof[0].fitness.values[0])